> Support Vector Machine

- Support Vector Machine은 모델이다.

# 1. 선형 SVM 분류

- 클래스가 선형으로 분류될 때
- 클래스 분류의 경계에 있는 샘플 = 서포트 벡터


## 1) 소프트 마진 분류

- 클래스 분류의 경계에 샘플이 없을 때 하드 마진 분류

    - 선형적으로 구분되어야 함
    - 이상치에 민감함

이 떄 생기는 이상치를 마진 오류로 보는데, 하드 마진 분류